In [1]:
import pandas as pd
from string import ascii_uppercase as alphabet # string que contiene el alfabeto
import pickle
from bs4 import BeautifulSoup
import requests # para solicitar conexiones web
from scipy.stats import poisson

# Recolección de datos con BeautifulSoup

In [2]:
all_tables = pd.read_html("https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup#Group_stage")

In [3]:
# A --> H
all_tables[12]

,Pos,Teamvte,Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,Qatar (H),0,0,0,0,0,0,0,0,Advance to knockout stage
1,2,Ecuador,0,0,0,0,0,0,0,0,Advance to knockout stage
2,3,Senegal,0,0,0,0,0,0,0,0,NaN
3,4,Netherlands,0,0,0,0,0,0,0,0,NaN


In [4]:
for letter, i in zip(alphabet, range(12, 68, 7)): # Compruebo que hace esto
    print(letter, i)

A 12
B 19
C 26
D 33
E 40
F 47
G 54
H 61


In [5]:
all_tables = pd.read_html("https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup#Group_stage")
dict_tables = {} # Diccionario que guardará todos los dataframes para cada grupo

for letter, i in zip(alphabet, range(12, 68, 7)):
    df = all_tables[i]
    df.rename(columns = {df.columns[1] : "Team"}, inplace = True) # Cambiamos el nombre de la columna Teamvte
    df.pop("Qualification")
    dict_tables[f"Group {letter}"] = df # {Group X : df}
    #print(df)
    #print(" ")

In [6]:
dict_tables["Group H"]

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Portugal,0,0,0,0,0,0,0,0
1,2,Ghana,0,0,0,0,0,0,0,0
2,3,Uruguay,0,0,0,0,0,0,0,0
3,4,South Korea,0,0,0,0,0,0,0,0


In [7]:
with open(r"--- Ruta donde se encuentra tu archivo ---", "wb") as output:
    pickle.dump(dict_tables, output) # Envía toda la información de dict_tables al output

In [8]:
years = [year for year in range(1930, 2019, 4)] # Mundiales jugados hasta 2022

In [9]:
def get_Matches(Year):
    """
    
    """
    if Year != 2022: # Compruebo el año del mundial
        path_Year = f"https://en.wikipedia.org/wiki/{Year}_FIFA_World_Cup"
    else:
        path_Year = "https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup" # link a la web antes de jugarse el mundial 2022
        
    response = requests.get(path_Year) # solicitas el acceso al contenido de la web
    content = response.text # muestras el contenido 
    soup = BeautifulSoup(content, "lxml")
    
    matches = soup.find_all("div", class_ = "footballbox") # Guardo toda la información de todos los partidos
    
    home = []
    score = []
    away = []

    for match in matches:   # Itero sobre cada uno de los partidos y los guardo la info en listas 
        home.append(match.find("th", class_ = "fhome").get_text())
        score.append(match.find("th", class_ = "fscore").get_text())
        away.append(match.find("th", class_ = "faway").get_text())

    dict_matches = {"home": home,
                    "score" : score,
                    "away" : away}

    df_matches = pd.DataFrame(dict_matches)
    df_matches["year"] = Year

    return df_matches

In [10]:
# Historical data
fifa = [get_Matches(year) for year in years] # Guardo todos los dataframes en una lista
df_Fifa = pd.concat(fifa, ignore_index = True) # Concatenos todos los dataframes en uno sólo
df_Fifa.to_csv(r"--- Ruta de donde vas a guardar el archivo ---", index = False)

#print(get_Matches(1982))

# Fixture
df_fixture = get_Matches(2022)
df_fixture.to_csv(r"--- Ruta de donde vas a guardar el archivo ---", index = False)

# Limpieza de datos

In [19]:
df_historic_data = pd.read_csv("Fifa_Wordlcup_Historical_Data.csv") # Cargo el csv con todos los mundiales hasta el 2018
df_historic_data
df_fixture = pd.read_csv("Fifa_Wordlcup_Fixture.csv") # Estructura del mundial 2022

### Limpieza df_fixture

In [13]:
df_fixture["home"] = df_fixture["home"].str.strip() # ELimino posibles espacios en blancos al principio o al final 
df_fixture["away"] = df_fixture["away"].str.strip()

### Limpiando df_historical_data

In [20]:
df_historic_data.isnull() # Compruebo si faltan datos
df_historic_data.sort_values("year", inplace = True) # Ordeno el DF según el año
df_historic_data

,home,score,away,year
0,France,4–1,Mexico,1930
16,Uruguay,4–2,Argentina,1930
15,Uruguay,6–1,Yugoslavia,1930
14,Argentina,6–1,United States,1930
13,Paraguay,1–0,Belgium,1930
...,...,...,...,...
826,Brazil,2–0,Costa Rica,2018
827,Serbia,1–2,Switzerland,2018
828,Serbia,0–2,Brazil,2018
830,Germany,0–1,Mexico,2018


In [21]:
row_delete = df_historic_data[df_historic_data["score"].str.contains("w/o")].index # busco si algún partido no se jugó
row_delete

Index([36], dtype='int64')

In [22]:
df_historic_data.drop(index = row_delete, inplace = True) # Eliminamos la entrada del DF

In [9]:
df_historic_data[df_historic_data["score"].str.contains("w/o")]

,home,score,away,year


In [42]:
df_historic_data["score"].values # Miro los valores de la variable score

array(['4–1', '1–0', '3–0', '1–0', '6–3', '3–1', '2–1', '4–0', '3–1',
       '1–0', '4–0', '3–0', '3–0', '1–0', '6–1', '6–1', '4–2', '3–1',
       '4–2', '3–2', '7–1', '2–1', '3–2', '3–2 (a.e.t.)', '5–2', '2–1',
       '1–1 (a.e.t.)', '2–1', '3–2', '1–0', '1–0', '3–1', '3–2',
       '2–1 (a.e.t.)', '1–1 (a.e.t.)', '6–0', '3–3 (a.e.t.)', '3–1',
       '2–1 (a.e.t.)', '6–5 (a.e.t.)', '3–0 (a.e.t.)', '4–2', '2–1',
       '2–0', '8–0', '3–1', '1–1 (a.e.t.)', '2–1', '5–1', '2–1', '4–2',
       '4–2', '4–0', '3–0', '2–2', '4–1', '2–0', '2–1', '2–0', '3–1',
       '2–0', '1–0', '1–0', '5–2', '3–2', '2–2', '2–0', '8–0', '2–2',
       '7–1', '6–1', '3–2', '3–1', '2–1', '5–0', '1–0', '1–1 (a.e.t.)',
       '3–2', '4–1', '9–0', '8–3', '7–0', '7–2', '2–0', '1–0', '7–0',
       '5–0', '2–1', '4–4 (a.e.t.)', '4–1', '2–0', '4–1', '7–5', '4–2',
       '2–0', '4–2', '6–1', '4–2 (a.e.t.)', '3–1', '3–2', '1–3', '1–0',
       '2–2', '3–1', '2–2', '6–1', '2–1 (a.e.t.)', '7–3', '1–1', '3–2',
       '3–2', '

In [24]:
# df_historic_data[df_historic_data["score"].str.contains("a.e.")] 
# Filtro donde se encuentran los datos a limpiar
df_historic_data["score"] = df_historic_data["score"].str.replace("(a.e.t.)", "", regex = False)
df_historic_data["score"] = df_historic_data["score"].str.replace("(a.e.t./g.g.)", "", regex = False)
df_historic_data[df_historic_data["score"].str.contains("a.e.t.")]
#df_historic_data["score"].describe(include = "all")


,home,score,away,year


In [ ]:
data = {"Home" : ["team1", "tema2", "team3", "team4", "team5"],
        "Score" : ["2-1 (a.e.t.)", "0-3", "1-4 (a.e.t.)", "2-2", "3-0"],
        "Away" : ["t1", "t2", "t3", "t4", "t5"]
        }

data_df = pd.DataFrame(data)

#data_df[data_df["Score"].str.contains(r"[^\d-]", regex = True)]
data_df["Score"] = data_df["Score"].str.replace(r"[^\d-]", "", regex = True)
data_df["Score"].values
#data_df["Score"].str.split("-", expand = True)

In [ ]:
data = {"Home" : ["team1", "tema2", "team3", "team4", "team5"],
        "Score" : ["2-1", "0-3", "1-4", "2-2", "3-0"],
       "Away" : ["t1", "t2", "t3", "t4", "t5"]
        }

data_df = pd.DataFrame(data)
data_df

data_df["Score"].str.split("-", expand = True)

In [25]:
df_historic_data[df_historic_data["score"].str.contains(r"[^\d-]", regex = True)]
# "[^\d-]" encuentra carácteres que no sean dígitos o un -

,home,score,away,year
0,France,4–1,Mexico,1930
16,Uruguay,4–2,Argentina,1930
15,Uruguay,6–1,Yugoslavia,1930
14,Argentina,6–1,United States,1930
13,Paraguay,1–0,Belgium,1930
...,...,...,...,...
826,Brazil,2–0,Costa Rica,2018
827,Serbia,1–2,Switzerland,2018
828,Serbia,0–2,Brazil,2018
830,Germany,0–1,Mexico,2018


In [26]:
df_historic_data["home"] = df_historic_data["home"].str.strip() # ELimino posibles espacios en blancos al principio o al final 
df_historic_data["away"] = df_historic_data["away"].str.strip()

In [29]:
df_historic_data["score"].str.split("-", expand = True)

,0
0,4–1
16,4–2
15,6–1
14,6–1
13,1–0
...,...
826,2–0
827,1–2
828,0–2
830,0–1


In [ ]:
df_historic_data[["HomeGoals", "AwayGoals"]] = df_historic_data["score"].str.split(pat = "-", expand = True)

In [ ]:
df_historic_data.drop("score", axis = 1, inplace = True) # Eliminamos la columna score

In [ ]:
# df_historic_data.columns[0:3] = ["HomeTeam", "AwayTeam", "Year"]
df_historic_data.rename(columns = {"home" : "HomeTeam", 
                                   "away" : "Awayteam",
                                   "year" : "Year"}, inplace = True)

In [ ]:
df_historic_data.dtypes
df_historic_data = df_historic_data.astype({"HomeGoals" : int,
                                            "AwayGoals" : int})

# Exportar DataFrames limpios

In [ ]:
df_historic_data.to_csv(r"--- Ruta a donde vas a guardar el archivo ---", index = False)
df_fixture.to_csv(r"--- Ruta a donde vas a guardar el archivo ---", index = False)

# Calcular Team Strength

In [ ]:
#dict_table = pickle.load(open(r"C:\Users\nenit\OneDrive\Escritorio\Data Science\Proyectos personales\Python\Proyecto Mundial\dict_table.json", "rb"))
#df_historic_data = pd.read_csv(r"C:\Users\nenit\OneDrive\Escritorio\Data Science\Proyectos personales\Python\Proyecto Mundial\Clean_Fifa_Wordlcup_Historical_Data.csv")
#df_fixture = pd.read_csv(r"C:\Users\nenit\OneDrive\Escritorio\Data Science\Proyectos personales\Python\Proyecto Mundial\Clean_Fifa_Wordlcup_Fixture.csv")

In [ ]:
#dict_tables.keys()
# Dividimos el df en df_home y df_away
df_home = df_historic_data[["HomeTeam", "HomeGoals", "AwayGoals"]]
df_away = df_historic_data[["AwayTeam", "HomeGoals", "AwayGoals"]]

In [ ]:
# Renombrar las  para estandarizar
df_home.rename(columns = {"HomeTeam" : "Team", "HomeGoals" : "GoalsScored",
                          "AwayGoals" : "GoalsConceded"}, inplace = True)
df_away.rename(columns = {"AwayTeam" : "Team", "HomeGoals" : "GoalsConceded",
                          "AwayGoals" : "GoalsScored"}, inplace = True)

In [ ]:
# Concatenar df_home y df_away, hacer group by Team y calcular mean para obtener la fortaleza de cada equipo
df_Team_Strenght = pd.concat([df_home, df_away], ignore_index = True).groupby("Team").mean()


# Función predict_points

In [ ]:
def predict_points(home, away) -> tuple:
    """
    Disposición de Poisson para predecir el equipo ganador del partido

    inputs:
    home(str) : equipo local
    away(str) : equipo visitante

    return:
    tupla con los puntos de cada equipo o
    (0, 0)
    """
    if home in df_Team_Strenght.index and away in df_Team_Strenght.index:
        # goals_scored  goals_conceded
        lamb_home = df_Team_Strenght.at[home, "GoalsScored"] * df_Team_Strenght.at[away, "GoalsConceded"]
        lamb_away = df_Team_Strenght.at[away, "GoalsScored"] * df_Team_Strenght.at[home, "GoalsConceded"]
        prob_home, prob_away, prob_draw = 0, 0, 0 

        for x in range(0,11): # number of goals home team
            for y in range(0, 11): # number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)

                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                elif x < y: # else:
                    prob_away += p

        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
      
        return (points_home, points_away)
    
    else:
        return (0, 0)

## Testear la función

In [ ]:
predict_points("Argentina", "Mexico")

# Prediciendo el resultado del Mundial

## Fase de grupo

In [ ]:
# Dividimos el df_fixture en fases

df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [ ]:
# Actualizamos las tablas de cada grupo con los resultados que da la función preditc_points
for group in dict_tables:
    group_teams = dict_tables[group]["Team"].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48["home"].isin(group_teams)]
    
    for index, row in df_fixture_group_6.iterrows():
        home, away = row["home"], row["away"]
        points_home, points_away = predict_points(home, away)

        dict_tables[group].loc[dict_tables[group]["Team"] == home, "Pts"] += points_home
        dict_tables[group].loc[dict_tables[group]["Team"] == away, "Pts"] += points_away
    
    dict_tables[group] = dict_tables[group].sort_values("Pts", ascending = False).reset_index()
    dict_tables[group] = dict_tables[group][["Team", "Pts"]]
    dict_tables[group] = dict_tables[group].round(0)

In [ ]:
dict_tables["Group C"]

## Octavos

In [ ]:
df_fixture_knockout

In [ ]:
for group in dict_tables:
    # Obtenemos el 1 y 2 segundo mejor equipo de cada grupo
    group_winner = dict_tables[group].loc[0, "Team"]
    group_runner_up = dict_tables[group].loc[1, "Team"]
    # Actualizamos el DF con los dos mejores equipo de cada grupo
    df_fixture_knockout.replace({f"Winners {group}" : group_winner, 
                                 f"Runners_up {group}" : group_runner_up},
                                 inplace = True)

df_fixture_knockout["Winner"] = "?"
df_fixture_knockout

In [ ]:
def get_winner(df_fixture_updated) -> pd.DataFrame:
    """
    Función que predice que equipo va a ser el ganador del mundial

    inputs:
    df_fixture_updated (DF) : dataframe con los dos equipos que llegarían a la final

    return:
    df_fixture_updated (DF) : mismo DF pero se le modifica la columna "winner"
    """
    for index, row in df_fixture_updated.iterrows():
        home, away = row["home"], row["away"]
        home_points, away_points = predict_points(home, away)

        if home_points > away_points:
            winner = home

        elif home_points < away_points:
            winner = away

        df_fixture_updated.loc[index, "Winner"] = winner

    return df_fixture_updated

In [ ]:
get_winner(df_fixture_knockout) # Jugamos los partidos de los octavos

## Cuartos de final

In [ ]:
df_fixture_quarter

In [ ]:
def update_table(df_fixture_1, df_fixture_2) -> pd.DataFrame:
    """
    Función que actualiza el df_fixture_quarter con los resultados de
    los partidos de octavos

    inputs:
    df_fixture_1 (DF) : DF con los partidos de octavos ya jugados
    df_fixture_2 (DF) : DF con los partidos de cuartos sin jugar

    return:
    df_fixture_2 actualizado con los resultados de los partidos de octavos
    """
    for index, row in df_fixture_1.iterrows():
        winner = df_fixture_1.loc[index, "winner"]
        match = df_fixture_1.loc[index, "score"]

        df_fixture_2.replace({f"Winner {match}" : winner},
                             inplace = True)
        
    df_fixture_2["winner"] = "?"

    return df_fixture_2

In [ ]:
update_table(df_fixture_knockout, df_fixture_quarter) # Actualizamos el DF de los cuartos de final

In [ ]:
get_winner(df_fixture_quarter) # Jugamos los partidos de los cuartos de final

## Semifinal

In [ ]:
update_table(df_fixture_quarter, df_fixture_semi) # Actualizamos el DF de la semifinal

In [ ]:
get_winner(df_fixture_semi) # Jugamos los partidos de la semifinal

## Final

In [ ]:
update_table(df_fixture_semi, df_fixture_final) # Actualizamos el DF de la final

In [ ]:
df_third_place = df_fixture_final[0].copy() # Obtenemos la estructura para el tercer puesto

In [ ]:
# Filtro para encontrar a los perdedores de ambos partidos
loser1 = df_fixture_semi[df_fixture_semi["home"][0] or df_fixture_semi["away"][0] != df_fixture_semi["winner"][0]]
loser2 = df_fixture_semi[df_fixture_semi["home"][1] or df_fixture_semi["away"][1] != df_fixture_semi["winner"][1]]
df_third_place["home"] = loser1
df_third_place["away"] = loser2

In [ ]:
get_winner(df_third_place) # Tercer puesto

In [ ]:
get_winner(df_fixture_final) # Ganador del Mundial!!